In [0]:
#### configura conexão
spark.conf.set("fs.azure.account.auth.type", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id", "8f8d5ab6-40c4-4c8c-961a-b137f487ecd4")
spark.conf.set("fs.azure.account.oauth2.client.secret", dbutils.secrets.get(scope="brewdatakvgbdev",key="brewdat-spn-gb-it-d-bees-fw-s-rw"))
spark.conf.set("fs.azure.account.oauth2.client.endpoint", "https://login.microsoftonline.com/cef04b19-7776-4a94-b89b-375c77a8f936/oauth2/token")


In [0]:
#### configura conexão le arquivos utilizando spark 
account = spark.read.format("csv").option("header","true").load('abfss://bees-plz@brewdatblobsagbdev.dfs.core.windows.net/maz-co-teste/prov/part-00000-tid-4727103262398151985-402ef528-4c81-4834-b4ad-541c0da72cd8-5636803-1-c000.csv')

month = spark.read.format("csv").option("header","true").load('abfss://bees-plz@brewdatblobsagbdev.dfs.core.windows.net/maz-co-teste/prov/part-00000-tid-2126202298789004890-eb1868ce-508e-4d67-b1bc-baccf22fb7c9-5632439-1-c000.csv'
)

time = spark.read.format("csv").option("header","true").load('abfss://bees-plz@brewdatblobsagbdev.dfs.core.windows.net/maz-co-teste/prov/part-00000-tid-4544860247627019718-dd4a1ae5-7a30-4e83-9d30-84e076e5de86-5619754-1-c000.csv'
)

year = spark.read.format("csv").option("header","true").load('abfss://bees-plz@brewdatblobsagbdev.dfs.core.windows.net/maz-co-teste/prov/part-00000-tid-7069962764778277724-d711ec89-a3ba-4871-815d-048009699f39-5635522-1-c000.csv'
)

pix_movements = spark.read.format("csv").option("header","true").load('abfss://bees-plz@brewdatblobsagbdev.dfs.core.windows.net/maz-co-teste/prov/part-00000-tid-8452669778079327534-8b8fcb3c-39d4-459f-81e8-c72e386ef3bd-5574198-1-c000.csv'
)

transfer_ins = spark.read.format("csv").option("header","true").load('abfss://bees-plz@brewdatblobsagbdev.dfs.core.windows.net/maz-co-teste/prov/part-00000-tid-3029698034815685249-7fc41d58-c231-4e33-ac38-4f87141a5780-5573069-1-c000.csv'
)

transfer_outs = spark.read.format("csv").option("header","true").load('abfss://bees-plz@brewdatblobsagbdev.dfs.core.windows.net/maz-co-teste/prov/part-00000-tid-8080996519800486434-f8e56525-3cc5-44c9-9fad-7ebeefc4bbb9-5573447-1-c000.csv'
)


In [0]:
#### cria tabela temporaria dos dataFrame
account.registerTempTable('tab_account')
month.registerTempTable('tab_month')
time.registerTempTable('tab_time')
year.registerTempTable('tab_year')
pix_movements.registerTempTable('tab_pix_movements')
transfer_ins.registerTempTable('tab_transfer_ins')
transfer_outs.registerTempTable('tab_transfer_outs')


In [0]:
### cria DataFrame com todos os lançamentos 
resultado =  spark.sql('''
select a.account_id, lpad(month, 2,'0') as month, year, sum(debito) as debito, sum(credito) as credito
from tab_account a 
left join 
( (
  select a.account_id, c.action_month as month , d.action_year as year 
  , Case When a.in_or_out = 'pix_out' then cast(a.pix_amount as decimal(18,2)) end as debito , Case When a.in_or_out = 'pix_in' then             cast(a.pix_amount as decimal(18,2)) end as credito

  from tab_pix_movements a 
  left join tab_time as b
  on a.pix_completed_at = b.time_id
  left join tab_month as c 
  on b.month_id = c.month_id
  left join tab_year as d 
  on b.year_id = d.year_id
  where a.status = 'completed'
  ) 
  union all
  (
  select a.account_id, c.action_month, d.action_year, 0 as debito , cast(a.amount as decimal(18,2)) as credito
  from tab_transfer_ins a 
  left join tab_time as b
  on a.transaction_completed_at = b.time_id
  left join tab_month as c 
  on b.month_id = c.month_id
  left join tab_year as d 
  on b.year_id = d.year_id
  where a.status = 'completed'
  ) 
    union all
  (
  select a.account_id, c.action_month, d.action_year, cast(a.amount as decimal(18,2)) as debito , 0 as credito
  from tab_transfer_outs a 
  left join tab_time as b
  on a.transaction_completed_at = b.time_id
  left join tab_month as c 
  on b.month_id = c.month_id
  left join tab_year as d 
  on b.year_id = d.year_id
  where a.status = 'completed'
  )
  ) b

on a.account_id = b.account_id 
where  a.account_id    = '1000667155163612544'
group by a.account_id, month, year 
order by a.account_id, lpad(month, 2,'0'), year ''')

In [0]:
#### cria tabela temporaria do resultado
resultado.registerTempTable('tab_resultado')

In [0]:
#### gera dataframe final e arquivo parquet
arquivo_parquet =  spark.sql('''
select account_id, month, year
, cast(sum((Balance_credito - Balance_debito) - ( credito - debito ) ) as decimal(18,2)) as saldo_inicial
, sum(credito) as credito 
, sum(debito) as debito
, cast(sum(((Balance_credito - Balance_debito) - ( credito - debito ) ) + credito - debito ) as decimal(18,2)) as saldo_final

from 
(
select account_id, month, year, debito, credito

, sum(Debito) OVER( ORDER BY account_id ROWS BETWEEN UNBOUNDED PRECEDING  AND CURRENT ROW) as Balance_debito
, sum(Credito) OVER( ORDER BY account_id ROWS BETWEEN UNBOUNDED PRECEDING  AND CURRENT ROW) as Balance_credito
from tab_resultado
)
group by account_id, month, year ''' )

arquivo_parquet.write.format('parquet').option("header","true").save('abfss://bees-plz@brewdatblobsagbdev.dfs.core.windows.net/maz-co-teste/prov/arquivo.parquet')